Goal: improve runtime efficiency and memory efficiency of pipeline
1. Need ROOT file from hepmc file (GOOD)
1. Process ROOT file to csv (check) (extractCellID)
1. Generate SiPM Output (needs work) (momentum_prediction_util)
1. Train NN (GOOD)


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plot
from  pandas import read_csv as pd_read_csv
from time_res_util import get_compiled_NF_model
from momentum_prediction_util import SiPMSignalProcessor
import datetime
def print_w_time(message):
    current_time = datetime.datetime.now().strftime('%H:%M:%S')
    print(f"{current_time} {message}")
import torch
import time
from IPython.display import clear_output
from collections import defaultdict

Using device cuda:0


In [2]:
# inputProcessedData = "./data/processed_data/dev_branch_100events.csv"
inputProcessedData = "./data/processed_data/dev_branch_50events.csv"
processed_data = pd_read_csv(inputProcessedData)
model_compile = get_compiled_NF_model()

/hpc/group/vossenlab/rck32/ML_venv/lib64/python3.9/site-packages/normflows/core.py:213: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.load(path))


In [3]:
begin = time.time()
batch_size=50000
normalizing_flow =model_compile
device='cuda'
pixel_threshold = 3
out_columns = ['event_idx','stave_idx','layer_idx','segment_idx','trueID','truePID','hitID','hitPID','P','Theta','Phi','strip_x','strip_y','strip_z','Charge1','Time1','Charge2','Time2']
rows = []
processor = SiPMSignalProcessor()
normflow_input = []
num_pixel_list = ["num_pixels_high_z","num_pixels_low_z"]
print_w_time("Processing data in generateSiPMOutput...")
got_row_indexes = False
labels = []
for row_idx, row in processed_data.iterrows():
    for SiPM_idx in range(2):
        num_pixel_tag = num_pixel_list[SiPM_idx] #get the name of the number of pixels for this SiPM
        #create a new row for each of the photons/pixels
        row_copy = row.copy()
        row_copy['SiPM_idx'] = SiPM_idx
        if(not got_row_indexes):
            labels = row_copy.index.to_list()
            got_row_indexes = True
        normflow_input.append(torch.tensor(row_copy.values,dtype=torch.float32).repeat(int(row_copy[num_pixel_tag]), 1))
#Create a dict like{label : index in row}
label_dict = {}
for i in range(len(labels)):
    label_dict[labels[i]] = i

normflow_input_tensor = torch.cat(normflow_input)

data = []
print_w_time(f"starting sampling")
begin = time.time()
#     for i in tqdm(range(0, len(normflow_input_tensor), batch_size)):
for i in range(0, len(normflow_input_tensor), batch_size):
    print_w_time(f"Starting batch # {(i / batch_size) + 1} / {int(np.ceil(len(normflow_input_tensor) / batch_size))}")
    batch_end = min(i + batch_size, len(normflow_input_tensor))
    batch_rows = normflow_input_tensor[i:batch_end]
    context_indexes = [label_dict["z_pos"],label_dict["hittheta"],label_dict["hitmomentum"]]
    time_index = label_dict["time"]
    batch_context = batch_rows[:,context_indexes].to(device)
    batch_particle_hit_times = batch_rows[:,time_index]

    with torch.no_grad():
        samples = abs(normalizing_flow.sample(num_samples=len(batch_context), context=batch_context)[0]).squeeze(1).cpu() + batch_particle_hit_times
    batch_combined_data = torch.cat((batch_rows,samples.unsqueeze(-1)),dim = 1)
    data.extend(batch_combined_data)
end = time.time()
print_w_time(f"sampling took {(end - begin) / 60} minutes")
print_w_time("creating df")
labels.append("photon_time")
row_df = pd.DataFrame(data,columns = labels)
end = time.time()
print(f"This cell took {end - begin} seconds")

21:58:39 Processing data in generateSiPMOutput...
21:58:43 starting sampling
21:58:43 Starting batch # 1.0 / 8
21:58:45 Starting batch # 2.0 / 8
21:58:46 Starting batch # 3.0 / 8
21:58:46 Starting batch # 4.0 / 8
21:58:48 Starting batch # 5.0 / 8
21:58:49 Starting batch # 6.0 / 8
21:58:50 Starting batch # 7.0 / 8
21:58:51 Starting batch # 8.0 / 8
21:58:52 sampling took 0.14970502058664958 minutes
21:58:52 creating df
This cell took 50.76135444641113 seconds


#### Original

In [ ]:
begin = time.time()
print_w_time("Beginning pulse process")
running_index = 0
for (event_idx, stave_idx, layer_idx, segment_idx),group in row_df.groupby(['event_idx', 'stave_idx', 'layer_idx','segment_idx']):
#         print(f"Starting row # {running_index}\nevent #{event_idx}, stave #{stave_idx}, layer #{layer_idx}, segment #{segment_idx}")
    charge_times = torch.tensor([[0.0,0.0],[0.0,0.0]])
    set_event_details = False
    trigger = False
    trueID_list_len_max = -1
    for SiPM_idx, SiPM_group in group.groupby(['SiPM_idx']):
        SiPM_idx_int = int(SiPM_idx[0].item())
        #need to see if we get more than 1 hit in a segment - greg says to label this as noise
        trueID_list_len = len(set(SiPM_group["trueID"]))
        trueID_list_len_max = max(trueID_list_len,trueID_list_len_max)
        photon_times = torch.tensor(sorted(SiPM_group['photon_time'])) * 10 **(-9)
        #get relative times
        if(len(photon_times) > 0):
            #calculate time and charge
            time_arr,waveform = processor.generate_waveform(photon_times)
            timing = processor.get_pulse_timing(waveform,threshold = pixel_threshold)
            if(timing is not None):
                curr_charge = processor.integrate_charge(waveform) * 1e6
                curr_timing = timing * 1e8
                charge_times[SiPM_idx_int][0] = processor.integrate_charge(waveform) * 1e6
                charge_times[SiPM_idx_int][1] = timing * 1e8
#                                 print(f"SiPM idx {SiPM_idx} triggered, (time,charge) : ({curr_timing},{curr_charge})")
                trigger = True
            else: #no trigger, don't set details yet
                continue
            if(not set_event_details):
                #take the 0th one - if there are multiple, then these are noise anyways
                P = SiPM_group['truemomentum'].iloc[0]
                trueID = SiPM_group['trueID'].iloc[0]
                truePID = SiPM_group['truePID'].iloc[0]
                hitID = SiPM_group['hitID'].iloc[0]
                hitPID = SiPM_group['hitPID'].iloc[0]
                theta = SiPM_group['truetheta'].iloc[0]
                phi = SiPM_group['truephi'].iloc[0]
                strip_x = SiPM_group['strip_x'].iloc[0]
                strip_y = SiPM_group['strip_y'].iloc[0]
                strip_z = SiPM_group['strip_z'].iloc[0]
                set_event_details = True
        else: #no photons, no data
            continue
    if(not set_event_details):
        continue
    if (not trigger):
        continue;
    noise = False
    if(trueID_list_len_max > 1):
        noise = True
    if(not noise):
        if(trueID_dict[event_idx][trueID.item()] == -1):
            trueID_dict[event_idx][trueID.item()] = trueID_dict_running_idx
            trueID_dict_running_idx += 1
        translated_trueID = trueID_dict[event_idx][trueID.item()]
    else:
        translated_trueID = -1
    new_row = { 
       out_columns[0] : event_idx,
       out_columns[1] : stave_idx,
       out_columns[2] : layer_idx,
       out_columns[3] : segment_idx,
       out_columns[4] : translated_trueID, 
       "original_trueID" : trueID.item(), 
       out_columns[5] : truePID.item(), 
       out_columns[6] : hitID.item(),
       out_columns[7] : hitPID.item(),
       out_columns[8] : P.item(), 
       out_columns[9] : theta.item(), 
      out_columns[10] : phi.item(), 
      out_columns[11] : strip_z.item(), 
      out_columns[12] : strip_x.item(), 
      out_columns[13] : strip_y.item(), 
      out_columns[14] : charge_times[0,0].item(), 
      out_columns[15] : charge_times[0,1].item(), 
      out_columns[16] : charge_times[1,0].item(), 
      out_columns[17] : charge_times[1,1].item(),
    }
    rows.append(new_row)
    running_index += 1
df = pd.DataFrame(rows,columns = out_columns)
end = time.time()
print(f"Original Method took {(end - begin)} seconds")

21:55:52 Beginning pulse process


#### New

In [4]:
begin = time.time()
results = []

trueID_dict = defaultdict(lambda: defaultdict(lambda: -1))
trueID_dict_running_idx = 0


# Get the column values as numpy arrays for faster access
event_indices = row_df['event_idx'].values
stave_indices = row_df['stave_idx'].values
layer_indices = row_df['layer_idx'].values
segment_indices = row_df['segment_idx'].values

current_indices = [-1, -1, -1, -1]  # [event, stave, layer, segment]
segment_start = 0
running_segment_idx = 0
# loop over all hits in df
for i in range(len(row_df) + 1):
    # Check if we've reached a new segment or the end of the dataset
    if i == len(row_df) or (
        event_indices[i] != current_indices[0] or
        stave_indices[i] != current_indices[1] or
        layer_indices[i] != current_indices[2] or
        segment_indices[i] != current_indices[3]
    ):
        # Process the previous segment if it exists
        if current_indices[0] != -1:
            #get all hits in segment
            segment_data = row_df.iloc[segment_start:i]
            running_segment_idx += 1
#             print(f"processing segment #{running_segment_idx}")
            clear_output(wait=True)
            # Your processing logic here
            
            
            charge_times = torch.tensor([[0.0,0.0],[0.0,0.0]])
            set_event_details = False
            trigger = False
            trueID_list_len_max = -1
            for SiPM_idx, SiPM_group in segment_data.groupby(['SiPM_idx']):
                SiPM_idx_int = int(SiPM_idx[0].item())
                #need to see if we get more than 1 hit in a segment - greg says to label this as noise
                trueID_list_len = len(set(SiPM_group["trueID"].apply(lambda x: x.item())))
                trueID_list_len_max = max(trueID_list_len,trueID_list_len_max)
                photon_times = torch.tensor(SiPM_group["photon_time"].to_list()) * 10 **(-9)
                #get relative times
                if(len(photon_times) > 0):
                    #calculate time and charge
                    time_arr,waveform = processor.generate_waveform(photon_times)
                    timing = processor.get_pulse_timing(waveform,threshold = pixel_threshold)
                    if(timing is not None):
                        curr_charge = processor.integrate_charge(waveform) * 1e6
                        curr_timing = timing * 1e8
                        charge_times[SiPM_idx_int][0] = processor.integrate_charge(waveform) * 1e6
                        charge_times[SiPM_idx_int][1] = timing * 1e8
        #                                 print(f"SiPM idx {SiPM_idx} triggered, (time,charge) : ({curr_timing},{curr_charge})")
                        trigger = True
                    else: #no trigger, don't set details yet
                        continue
                    if(not set_event_details):
                        #take the 0th one - if there are multiple, then these are noise anyways
                        P = SiPM_group['truemomentum'].iloc[0]
                        trueID = SiPM_group['trueID'].iloc[0]
                        truePID = SiPM_group['truePID'].iloc[0]
                        hitID = SiPM_group['hitID'].iloc[0]
                        hitPID = SiPM_group['hitPID'].iloc[0]
                        theta = SiPM_group['truetheta'].iloc[0]
                        phi = SiPM_group['truephi'].iloc[0]
                        strip_x = SiPM_group['strip_x'].iloc[0]
                        strip_y = SiPM_group['strip_y'].iloc[0]
                        strip_z = SiPM_group['strip_z'].iloc[0]
                        set_event_details = True
                else: #no photons, no data
                    continue
            noise = False
            if(trueID_list_len_max > 1):
                noise = True
            if(not noise):
                if(trueID_dict[current_indices[0]][trueID.item()] == -1):
                    trueID_dict[current_indices[0]][trueID.item()] = trueID_dict_running_idx
                    trueID_dict_running_idx += 1
                translated_trueID = trueID_dict[current_indices[0]][trueID.item()]
            else:
                translated_trueID = -1
            if(set_event_details) and (trigger):
#                 print("Adding new row")
                new_row = { 
                   out_columns[0] : current_indices[0],
                   out_columns[1] : current_indices[1],
                   out_columns[2] : current_indices[2],
                   out_columns[3] : current_indices[3],
                   out_columns[4] : translated_trueID, 
                   "original_trueID" : trueID.item(), 
                   out_columns[5] : truePID.item(), 
                   out_columns[6] : hitID.item(),
                   out_columns[7] : hitPID.item(),
                   out_columns[8] : P.item(), 
                   out_columns[9] : theta.item(), 
                  out_columns[10] : phi.item(), 
                  out_columns[11] : strip_z.item(), 
                  out_columns[12] : strip_x.item(), 
                  out_columns[13] : strip_y.item(), 
                  out_columns[14] : charge_times[0,0].item(), 
                  out_columns[15] : charge_times[0,1].item(), 
                  out_columns[16] : charge_times[1,0].item(), 
                  out_columns[17] : charge_times[1,1].item(),
                }
                results.append(new_row)
        if i < len(row_df):
            # Update indices for the new segment
            current_indices = [
                event_indices[i],
                stave_indices[i],
                layer_indices[i],
                segment_indices[i]
            ]
            #save the index of the beginning of segment so we know
            #which hits are in this segment
            segment_start = i

result_df = pd.DataFrame(results)
print(f"Finished")
end = time.time()
print(f"New method took {end - begin} seconds")

Finished
New method took 70.85411930084229 seconds


In [5]:
result_df

,event_idx,stave_idx,layer_idx,segment_idx,trueID,original_trueID,truePID,hitID,hitPID,P,Theta,Phi,strip_x,strip_y,strip_z,Charge1,Time1,Charge2,Time2
0,tensor(14.),tensor(0.),tensor(0.),tensor(9.),0,0.0,130.0,14.0,1.000220e+09,0.150235,101.730644,-58.716457,96.910843,-159.592148,0.0,0.101561,3.589461,0.330018,3.554926
1,tensor(14.),tensor(0.),tensor(0.),tensor(11.),1,0.0,130.0,9.0,2.112000e+03,0.150235,101.730644,-58.716457,101.231018,-155.271973,0.0,0.000000,0.000000,0.139634,3.669460
2,tensor(14.),tensor(0.),tensor(0.),tensor(10.),2,0.0,130.0,45.0,1.100000e+01,0.150235,101.730644,-58.716457,99.070930,-157.432053,0.0,0.000000,0.000000,0.101545,3.569203
3,tensor(14.),tensor(0.),tensor(0.),tensor(8.),3,0.0,130.0,31.0,2.112000e+03,0.150235,101.730644,-58.716457,94.750755,-161.752228,0.0,0.000000,0.000000,0.088839,3.590492
4,tensor(14.),tensor(0.),tensor(0.),tensor(5.),4,0.0,130.0,54.0,1.000241e+09,0.150235,101.730644,-58.716457,88.270493,-168.232498,0.0,0.050762,3.869566,0.241107,3.698899
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,tensor(30.),tensor(5.),tensor(11.),tensor(25.),251,0.0,130.0,394.0,2.112000e+03,1.220183,91.994919,89.898590,31.580360,265.744995,0.0,0.000000,0.000000,0.050807,2.942153
191,tensor(30.),tensor(5.),tensor(11.),tensor(36.),252,0.0,130.0,460.0,1.000251e+09,1.220183,91.994919,89.898590,-1.500000,265.744995,0.0,0.127055,2.202142,0.050825,2.145680
192,tensor(30.),tensor(5.),tensor(11.),tensor(35.),253,0.0,130.0,465.0,1.100000e+01,1.220183,91.994919,89.898590,1.507306,265.744995,0.0,0.063528,2.236315,0.000000,0.000000
193,tensor(30.),tensor(5.),tensor(12.),tensor(37.),254,0.0,130.0,462.0,1.100000e+01,1.220183,91.994919,89.898590,-1.500000,273.415009,0.0,0.254108,2.229747,0.050819,2.365991


In [ ]:
result_df